<a href="https://colab.research.google.com/github/lehai0609/KagglePlayground/blob/main/Playground_S5E7_Improving_0_973279.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Overview

In this notebook, I'm doing EDA for the dataset and perform a baseline model based on EDA analysis.
It's a very standardized process, just like the dataset provided in this month's challenge

In [27]:
# prompt: Read train.csv & test.csv from folder: /content/drive/MyDrive/Kaggle/Playground S5E7

import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Playground S5E7/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Kaggle/Playground S5E7/test.csv')

In [28]:
train_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [29]:
test_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0


# III. TARGET ENCODED APPROACH

In [30]:
# Verify GPU availability (after restart)
import cupy as cp
import cudf as cd

# Check GPU info
device = cp.cuda.Device()
print(f"GPU ID: {device.id}")
print(f"GPU Memory: {device.mem_info[1] / 1024**3:.1f} GB total")
print(f"GPU Memory Available: {device.mem_info[0] / 1024**3:.1f} GB free")

# Test cuDF
try:
    test_cudf = cd.DataFrame({'test': [1, 2, 3]})
    print("✅ cuDF working correctly")
    print(f"cuDF version: {cd.__version__}")
except Exception as e:
    print(f"❌ cuDF error: {e}")

# Test cuML
try:
    from cuml.preprocessing import LabelEncoder
    print("✅ cuML working correctly")
except Exception as e:
    print(f"❌ cuML error: {e}")

# Test CuPy
try:
    test_array = cp.array([1, 2, 3])
    print("✅ CuPy working correctly")
    print(f"CuPy version: {cp.__version__}")
except Exception as e:
    print(f"❌ CuPy error: {e}")

GPU ID: 0
GPU Memory: 14.7 GB total
GPU Memory Available: 14.6 GB free
✅ cuDF working correctly
cuDF version: 25.02.01
✅ cuML working correctly
✅ CuPy working correctly
CuPy version: 13.3.0


In [31]:
import cudf as cd
import cupy as cp
import pandas as pd
import numpy as np
from cuml.model_selection import train_test_split
from cuml.preprocessing import LabelEncoder as cuLabelEncoder
from cuml.preprocessing import SimpleImputer as cuSimpleImputer
from cuml.metrics import roc_auc_score as cu_roc_auc_score
from cuml.ensemble import RandomForestClassifier as cuRandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print(f"GPU memory: {cp.cuda.Device().mem_info}")
print("RAPIDS cuDF and cuML loaded successfully")

# Define constants
NUMERICAL_FEATURES = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
CATEGORICAL_FEATURES = ['Stage_fear', 'Drained_after_socializing']
FEATURES = NUMERICAL_FEATURES + CATEGORICAL_FEATURES
STATS = ["mean", "count", "std"]
STATS2 = ["mean", "count"]
FOLDS = 5

print("Constants defined successfully")

GPU memory: (15680536576, 15828320256)
RAPIDS cuDF and cuML loaded successfully
Constants defined successfully


In [32]:
test_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0


In [33]:
# prompt: Impute all null values in numerical features by fillna of -1

for col in NUMERICAL_FEATURES:
  train_df[col] = train_df[col].fillna(-1)
  test_df[col] = test_df[col].fillna(-1)

print("Null values in numerical features imputed with -1.")

Null values in numerical features imputed with -1.


In [34]:
# Check for problematic data types that cuDF might not support
print("Checking for unsupported data types...")

def check_dataframe_compatibility(df, name):
    print(f"\n--- {name} ---")
    for col in df.columns:
        dtype = df[col].dtype
        unique_vals = df[col].nunique()
        nulls = df[col].isnull().sum()
        print(f"{col}: {dtype} | Unique: {unique_vals} | Nulls: {nulls}")

        # Check for object columns with mixed types
        if dtype == 'object':
            sample_values = df[col].dropna().head(10).tolist()
            print(f"  Sample values: {sample_values}")

check_dataframe_compatibility(train_df, "train_df")
check_dataframe_compatibility(test_df, "test_df")

Checking for unsupported data types...

--- train_df ---
id: int64 | Unique: 18524 | Nulls: 0
Time_spent_Alone: float64 | Unique: 13 | Nulls: 0
Stage_fear: object | Unique: 2 | Nulls: 1893
  Sample values: ['No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No']
Social_event_attendance: float64 | Unique: 12 | Nulls: 0
Going_outside: float64 | Unique: 9 | Nulls: 0
Drained_after_socializing: object | Unique: 2 | Nulls: 1149
  Sample values: ['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No']
Friends_circle_size: float64 | Unique: 17 | Nulls: 0
Post_frequency: float64 | Unique: 12 | Nulls: 0
Personality: object | Unique: 2 | Nulls: 0
  Sample values: ['Extrovert', 'Extrovert', 'Introvert', 'Extrovert', 'Extrovert', 'Extrovert', 'Extrovert', 'Extrovert', 'Introvert', 'Extrovert']

--- test_df ---
id: int64 | Unique: 6175 | Nulls: 0
Time_spent_Alone: float64 | Unique: 13 | Nulls: 0
Stage_fear: object | Unique: 2 | Nulls: 598
  Sample values: ['No', 'Yes', 'No', 'No', 'Yes', '

In [35]:
# Convert pandas DataFrames to cuDF for GPU acceleration
train_gpu = cd.from_pandas(train_df)
test_gpu = cd.from_pandas(test_df)

print("Data converted to GPU format")

# Prepare the data on GPU
train_clean = train_gpu.copy()
test_clean = test_gpu.copy()

# Convert target to binary using cuML LabelEncoder
le_target = cuLabelEncoder()
train_clean['Personality_binary'] = le_target.fit_transform(train_clean['Personality'])

# Check class distribution
print("Class distribution:")
personality_counts = train_clean['Personality'].value_counts()
print(f"Extrovert: {personality_counts.iloc[0] / len(train_clean):.3f}")
print(f"Introvert: {personality_counts.iloc[1] / len(train_clean):.3f}")

print(f"Training data shape: {train_clean.shape}")
print(f"Test data shape: {test_clean.shape}")

Data converted to GPU format
Class distribution:
Extrovert: 0.740
Introvert: 0.260
Training data shape: (18524, 10)
Test data shape: (6175, 8)


In [36]:
# Initialize cross-validation and result storage
# Note: StratifiedKFold from sklearn works with cuDF by converting indices
kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
kf2 = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=43)

# Storage for out-of-fold predictions and feature importance
oof_proba = cp.zeros(len(train_clean))
test_proba = cp.zeros(len(test_clean))
feature_importance = []

print("Cross-validation setup complete")

Cross-validation setup complete


In [37]:
# Helper function to handle cuDF groupby operations efficiently
def gpu_target_encode(df, group_col, target_col, stats_list, smoothing_factor=10):
    """Perform target encoding on GPU with smoothing"""
    grouped = df.groupby(group_col)[target_col].agg(stats_list)

    if 'count' in stats_list and 'mean' in stats_list:
        global_mean = df[target_col].mean()
        # Apply Bayesian smoothing
        smoothed_col = f"{group_col}_smoothed"
        grouped[smoothed_col] = (grouped['mean'] * grouped['count'] + global_mean * smoothing_factor) / (grouped['count'] + smoothing_factor)

    return grouped

print("Helper functions defined")

Helper functions defined


In [38]:
# Simplified Main nested cross-validation loop with GPU acceleration
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Constants (already defined)
NUMERICAL_FEATURES = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
CATEGORICAL_FEATURES = ['Stage_fear', 'Drained_after_socializing']
FEATURES = NUMERICAL_FEATURES + CATEGORICAL_FEATURES
STATS = ["mean", "count", "std"]
STATS2 = ["mean", "count"]
FOLDS = 5

# Initialize CV and storage
kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
kf2 = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=43)

oof_proba = np.zeros(len(train_df))
test_proba = np.zeros(len(test_df))

# Convert target to binary
train_df['Personality_binary'] = (train_df['Personality'] == 'Introvert').astype(int)

print("Starting nested cross-validation...")

# OUTER LOOP
for fold, (train_index, valid_index) in enumerate(kf.split(train_df, train_df['Personality_binary'])):
    print(f"\n=== FOLD {fold + 1} ===")

    # Split data for this fold
    X_train = train_df.iloc[train_index][FEATURES + ['Personality_binary']].copy()
    X_valid = train_df.iloc[valid_index][FEATURES].copy()
    X_test = test_df[FEATURES].copy()

    y_train = X_train['Personality_binary']
    y_valid = train_df.iloc[valid_index]['Personality_binary']

    print(f"Train size: {len(X_train)}, Valid size: {len(X_valid)}")

    # Handle missing values FIRST (critical for cuDF compatibility)
    # Numerical features: median imputation
    num_imputer = SimpleImputer(strategy='median')
    X_train[NUMERICAL_FEATURES] = num_imputer.fit_transform(X_train[NUMERICAL_FEATURES])
    X_valid[NUMERICAL_FEATURES] = num_imputer.transform(X_valid[NUMERICAL_FEATURES])
    X_test[NUMERICAL_FEATURES] = num_imputer.transform(X_test[NUMERICAL_FEATURES])

    # Categorical features: mode imputation
    cat_imputer = SimpleImputer(strategy='most_frequent')
    X_train[CATEGORICAL_FEATURES] = cat_imputer.fit_transform(X_train[CATEGORICAL_FEATURES])
    X_valid[CATEGORICAL_FEATURES] = cat_imputer.transform(X_valid[CATEGORICAL_FEATURES])
    X_test[CATEGORICAL_FEATURES] = cat_imputer.transform(X_test[CATEGORICAL_FEATURES])

    # Initialize feature storage
    train_features = X_train[FEATURES].copy()
    valid_features = X_valid.copy()
    test_features = X_test.copy()

    # INNER LOOP for target-encoded features (prevents leakage)
    print("Starting inner loop for target encoding...")

    # Storage for inner loop features
    te1_features = {}  # Feature Set 1: Individual target encoding
    te2_features = {}  # Feature Set 2: Interaction target encoding

    for inner_fold, (train_idx2, valid_idx2) in enumerate(kf2.split(X_train, y_train)):
        X_train2 = X_train.iloc[train_idx2].copy()
        X_valid2 = X_train.iloc[valid_idx2].copy()

        # === FEATURE SET 1: Individual numerical target encoding ===
        for col in NUMERICAL_FEATURES:
            # Create bins (now safe since nulls are handled)
            X_train2[f'{col}_binned'] = pd.qcut(X_train2[col], q=10, duplicates='drop')

            # Calculate target statistics
            grouped_stats = X_train2.groupby(f'{col}_binned')['Personality_binary'].agg(STATS)
            grouped_stats.columns = [f"TE1_{col}_{s}" for s in STATS]

            # Add smoothing for small groups
            global_mean = X_train2['Personality_binary'].mean()
            smoothing_factor = 10
            grouped_stats[f'TE1_{col}_smoothed'] = (
                (grouped_stats[f'TE1_{col}_mean'] * grouped_stats[f'TE1_{col}_count'] +
                 global_mean * smoothing_factor) /
                (grouped_stats[f'TE1_{col}_count'] + smoothing_factor)
            )

            # Create bins for validation data using same boundaries
            X_valid2[f'{col}_binned'] = pd.cut(X_valid2[col], bins=X_train2[f'{col}_binned'].cat.categories.left.tolist() + [X_train2[f'{col}_binned'].cat.categories.right[-1]])

            # Apply encoding to validation data
            for stat in STATS + ['smoothed']:
                col_name = f'TE1_{col}_{stat}'
                mapped_values = X_valid2[f'{col}_binned'].map(grouped_stats[col_name])
                X_valid2[col_name] = pd.Series(mapped_values, dtype='float64').fillna(global_mean)

                # Store for later aggregation
                if col_name not in te1_features:
                    te1_features[col_name] = np.zeros(len(X_train))
                te1_features[col_name][valid_idx2] = X_valid2[col_name].values

        # === FEATURE SET 2: Interaction target encoding ===
        interactions = [
            ('Social_event_attendance', 'Time_spent_Alone'),
            ('Stage_fear', 'Drained_after_socializing'),
            ('Friends_circle_size', 'Going_outside')
        ]

        for feat1, feat2 in interactions:
            # Create interaction feature
            if feat1 in NUMERICAL_FEATURES:
                X_train2[f'{feat1}_bin'] = pd.qcut(X_train2[feat1], q=5, duplicates='drop')
                feat1_use = f'{feat1}_bin'
            else:
                feat1_use = feat1

            if feat2 in NUMERICAL_FEATURES:
                X_train2[f'{feat2}_bin'] = pd.qcut(X_train2[feat2], q=5, duplicates='drop')
                feat2_use = f'{feat2}_bin'
            else:
                feat2_use = feat2

            interaction_name = f'{feat1}_{feat2}_interaction'
            X_train2[interaction_name] = X_train2[feat1_use].astype(str) + '_' + X_train2[feat2_use].astype(str)

            # Target encode the interaction
            grouped_stats = X_train2.groupby(interaction_name)['Personality_binary'].agg(STATS2)
            grouped_stats.columns = [f"TE2_{interaction_name}_{s}" for s in STATS2]

            # Apply smoothing
            global_mean = X_train2['Personality_binary'].mean()
            grouped_stats[f'TE2_{interaction_name}_smoothed'] = (
                (grouped_stats[f'TE2_{interaction_name}_mean'] * grouped_stats[f'TE2_{interaction_name}_count'] +
                 global_mean * 5) /
                (grouped_stats[f'TE2_{interaction_name}_count'] + 5)
            )

            # Apply to validation data
            # Create same features for validation
            if feat1 in NUMERICAL_FEATURES:
                X_valid2[f'{feat1}_bin'] = pd.cut(X_valid2[feat1], bins=X_train2[f'{feat1}_bin'].cat.categories.left.tolist() + [X_train2[f'{feat1}_bin'].cat.categories.right[-1]])
            if feat2 in NUMERICAL_FEATURES:
                X_valid2[f'{feat2}_bin'] = pd.cut(X_valid2[feat2], bins=X_train2[f'{feat2}_bin'].cat.categories.left.tolist() + [X_train2[f'{feat2}_bin'].cat.categories.right[-1]])

            X_valid2[interaction_name] = X_valid2[feat1_use].astype(str) + '_' + X_valid2[feat2_use].astype(str)

            for stat in STATS2 + ['smoothed']:
                col_name = f'TE2_{interaction_name}_{stat}'
                mapped_values = X_valid2[interaction_name].map(grouped_stats[col_name])
                X_valid2[col_name] = pd.Series(mapped_values, dtype='float64').fillna(global_mean)

                # Store for later aggregation
                if col_name not in te2_features:
                    te2_features[col_name] = np.zeros(len(X_train))
                te2_features[col_name][valid_idx2] = X_valid2[col_name].values

    # Add inner loop features to training data
    for col_name, values in te1_features.items():
        train_features[col_name] = values
    for col_name, values in te2_features.items():
        train_features[col_name] = values

    print(f"Created {len(te1_features)} TE1 features and {len(te2_features)} TE2 features")

    # === FEATURE SET 1 & 2 for validation/test (using full training data) ===
    # Now create the same features for validation and test using full X_train

    # TE1 for validation/test
    for col in NUMERICAL_FEATURES:
        X_train[f'{col}_binned'] = pd.qcut(X_train[col], q=10, duplicates='drop')
        grouped_stats = X_train.groupby(f'{col}_binned')['Personality_binary'].agg(STATS)
        grouped_stats.columns = [f"TE1_{col}_{s}" for s in STATS]

        global_mean = X_train['Personality_binary'].mean()
        grouped_stats[f'TE1_{col}_smoothed'] = (
            (grouped_stats[f'TE1_{col}_mean'] * grouped_stats[f'TE1_{col}_count'] +
             global_mean * 10) /
            (grouped_stats[f'TE1_{col}_count'] + 10)
        )

        # Apply to validation
        X_valid[f'{col}_binned'] = pd.cut(X_valid[col], bins=X_train[f'{col}_binned'].cat.categories.left.tolist() + [X_train[f'{col}_binned'].cat.categories.right[-1]])
        # Apply to test
        X_test[f'{col}_binned'] = pd.cut(X_test[col], bins=X_train[f'{col}_binned'].cat.categories.left.tolist() + [X_train[f'{col}_binned'].cat.categories.right[-1]])

        for stat in STATS + ['smoothed']:
            col_name = f'TE1_{col}_{stat}'
            mapped_valid = X_valid[f'{col}_binned'].map(grouped_stats[col_name])
            mapped_test = X_test[f'{col}_binned'].map(grouped_stats[col_name])
            valid_features[col_name] = pd.Series(mapped_valid, dtype='float64').fillna(global_mean)
            test_features[col_name] = pd.Series(mapped_test, dtype='float64').fillna(global_mean)

    # TE2 for validation/test
    for feat1, feat2 in interactions:
        # Same interaction creation logic as inner loop
        if feat1 in NUMERICAL_FEATURES:
            X_train[f'{feat1}_bin'] = pd.qcut(X_train[feat1], q=5, duplicates='drop')
            feat1_use = f'{feat1}_bin'
        else:
            feat1_use = feat1

        if feat2 in NUMERICAL_FEATURES:
            X_train[f'{feat2}_bin'] = pd.qcut(X_train[feat2], q=5, duplicates='drop')
            feat2_use = f'{feat2}_bin'
        else:
            feat2_use = feat2

        interaction_name = f'{feat1}_{feat2}_interaction'
        X_train[interaction_name] = X_train[feat1_use].astype(str) + '_' + X_train[feat2_use].astype(str)

        grouped_stats = X_train.groupby(interaction_name)['Personality_binary'].agg(STATS2)
        grouped_stats.columns = [f"TE2_{interaction_name}_{s}" for s in STATS2]

        global_mean = X_train['Personality_binary'].mean()
        grouped_stats[f'TE2_{interaction_name}_smoothed'] = (
            (grouped_stats[f'TE2_{interaction_name}_mean'] * grouped_stats[f'TE2_{interaction_name}_count'] +
             global_mean * 5) /
            (grouped_stats[f'TE2_{interaction_name}_count'] + 5)
        )

        # Apply to validation and test
        for dataset, name in [(X_valid, 'valid'), (X_test, 'test')]:
            if feat1 in NUMERICAL_FEATURES:
                dataset[f'{feat1}_bin'] = pd.cut(dataset[feat1], bins=X_train[f'{feat1}_bin'].cat.categories.left.tolist() + [X_train[f'{feat1}_bin'].cat.categories.right[-1]])
            if feat2 in NUMERICAL_FEATURES:
                dataset[f'{feat2}_bin'] = pd.cut(dataset[feat2], bins=X_train[f'{feat2}_bin'].cat.categories.left.tolist() + [X_train[f'{feat2}_bin'].cat.categories.right[-1]])

            dataset[interaction_name] = dataset[feat1_use].astype(str) + '_' + dataset[feat2_use].astype(str)

            for stat in STATS2 + ['smoothed']:
                col_name = f'TE2_{interaction_name}_{stat}'
                if name == 'valid':
                    mapped_values = dataset[interaction_name].map(grouped_stats[col_name])
                    valid_features[col_name] = pd.Series(mapped_values, dtype='float64').fillna(global_mean)
                else:
                    mapped_values = dataset[interaction_name].map(grouped_stats[col_name])
                    test_features[col_name] = pd.Series(mapped_values, dtype='float64').fillna(global_mean)

    # === FEATURE SET 3: Non-target statistical features ===
    fe3_count = 0
    for cat_col in CATEGORICAL_FEATURES:
        for num_col in NUMERICAL_FEATURES:
            # Calculate statistics on training data
            grouped = X_train.groupby(cat_col)[num_col].agg(['mean', 'std', 'median'])
            grouped.columns = [f"FE3_{cat_col}_{num_col}_{s}" for s in ['mean', 'std', 'median']]

            # Apply to all datasets
            for stat in ['mean', 'std', 'median']:
                col_name = f"FE3_{cat_col}_{num_col}_{stat}"
                default_val = X_train[num_col].mean() if stat != 'std' else X_train[num_col].std()

                mapped_train = X_train[cat_col].map(grouped[col_name])
                mapped_valid = X_valid[cat_col].map(grouped[col_name])
                mapped_test = X_test[cat_col].map(grouped[col_name])

                train_features[col_name] = pd.Series(mapped_train, dtype='float64').fillna(default_val)
                valid_features[col_name] = pd.Series(mapped_valid, dtype='float64').fillna(default_val)
                test_features[col_name] = pd.Series(mapped_test, dtype='float64').fillna(default_val)
                fe3_count += 1

    print(f"Created {fe3_count} FE3 features")

    # Label encode categorical features
    from sklearn.preprocessing import LabelEncoder
    le_stage = LabelEncoder()
    le_drained = LabelEncoder()

    train_features['Stage_fear'] = le_stage.fit_transform(train_features['Stage_fear'])
    train_features['Drained_after_socializing'] = le_drained.fit_transform(train_features['Drained_after_socializing'])

    valid_features['Stage_fear'] = le_stage.transform(valid_features['Stage_fear'])
    valid_features['Drained_after_socializing'] = le_drained.transform(valid_features['Drained_after_socializing'])

    test_features['Stage_fear'] = le_stage.transform(test_features['Stage_fear'])
    test_features['Drained_after_socializing'] = le_drained.transform(test_features['Drained_after_socializing'])

    print(f"Total features: {train_features.shape[1]}")

    # Ensure consistent feature order
    feature_cols = train_features.columns.tolist()
    train_features = train_features[feature_cols]
    valid_features = valid_features[feature_cols]
    test_features = test_features[feature_cols]

    # Train XGBoost model
    scale_pos_weight = len(y_train[y_train==0]) / len(y_train[y_train==1])

    model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        tree_method='gpu_hist',  # GPU acceleration
        gpu_id=0,
        max_depth=6,
        learning_rate=0.02,
        n_estimators=1000,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        enable_categorical=False,  # We label encoded everything
        early_stopping_rounds=50
    )

    # Train with early stopping
    model.fit(
        train_features, y_train,
        eval_set=[(valid_features, y_valid)],
        verbose=False
    )

    # Make predictions
    valid_pred = model.predict_proba(valid_features)[:, 1]
    test_pred = model.predict_proba(test_features)[:, 1]

    # Store predictions
    oof_proba[valid_index] = valid_pred
    test_proba += test_pred / FOLDS

    # Calculate metrics
    valid_auc = roc_auc_score(y_valid, valid_pred)
    valid_f1 = f1_score(y_valid, (valid_pred > 0.5).astype(int))
    valid_acc = accuracy_score(y_valid, (valid_pred > 0.5).astype(int))

    print(f"Fold {fold + 1} - AUC: {valid_auc:.4f}, F1: {valid_f1:.4f}, Accuracy: {valid_acc:.4f}")

# Calculate overall CV performance
overall_auc = roc_auc_score(train_df['Personality_binary'], oof_proba)
overall_f1 = f1_score(train_df['Personality_binary'], (oof_proba > 0.5).astype(int))
overall_acc = accuracy_score(train_df['Personality_binary'], (oof_proba > 0.5).astype(int))

print(f"\n=== OVERALL CV PERFORMANCE ===")
print(f"AUC: {overall_auc:.4f}")
print(f"F1: {overall_f1:.4f}")
print(f"Accuracy: {overall_acc:.4f}")

# # Create submission
# test_predictions_binary = (test_proba > 0.5).astype(int)
# test_predictions = np.where(test_predictions_binary == 1, 'Introvert', 'Extrovert')

# submission_df = pd.DataFrame({
#     'id': test_df['id'],
#     'Personality': test_predictions
# })

# submission_df.to_csv('submission.csv', index=False)
# print("Submission saved!")

Starting nested cross-validation...

=== FOLD 1 ===
Train size: 14819, Valid size: 3705
Starting inner loop for target encoding...
Created 20 TE1 features and 9 TE2 features
Created 30 FE3 features
Total features: 66
Fold 1 - AUC: 0.9665, F1: 0.9418, Accuracy: 0.9698

=== FOLD 2 ===
Train size: 14819, Valid size: 3705
Starting inner loop for target encoding...
Created 20 TE1 features and 9 TE2 features
Created 30 FE3 features
Total features: 66
Fold 2 - AUC: 0.9633, F1: 0.9357, Accuracy: 0.9668

=== FOLD 3 ===
Train size: 14819, Valid size: 3705
Starting inner loop for target encoding...
Created 20 TE1 features and 9 TE2 features
Created 30 FE3 features
Total features: 66
Fold 3 - AUC: 0.9650, F1: 0.9315, Accuracy: 0.9646

=== FOLD 4 ===
Train size: 14819, Valid size: 3705
Starting inner loop for target encoding...
Created 20 TE1 features and 9 TE2 features
Created 30 FE3 features
Total features: 66
Fold 4 - AUC: 0.9645, F1: 0.9408, Accuracy: 0.9695

=== FOLD 5 ===
Train size: 14820, V